# TASK 4.9 PART # 1

**This notebook contains the following scripts:**

**1. Data Wrangling**

**2. Data Quality and Consistency Checks**

**3. Merge consumer data with rest of Instacart data**

**4. Export new merged data frame as PKL**

In [2]:
#import libraries

import pandas as pd
import numpy as np
import os

In [3]:
#define path

path = r'C:\Users\ddgal\OneDrive\Desktop\Career Foundry Tasks\Data Immersion\22-03-2023 Instacart Basket Analysis'

In [4]:
#import files

cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_agg.pkl'))

**1. Data Wrangling**

In [6]:
#check cust data set

cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [8]:
#check number of rows and columns

cust.shape

(206209, 10)

In [10]:
#drop unnecessary columns First Name and Surnam

df_cust = cust.drop(columns = ['First Name', 'Surnam'])

In [11]:
#check

df_cust.head()

,user_id,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


In [16]:
#change column names

df_cust.rename(columns = {'Gender' : 'gender'}, inplace = True)
df_cust.rename(columns = {'n_dependants' : 'num_of_children'}, inplace = True)
df_cust.rename(columns = {'STATE' : 'state'}, inplace = True)
df_cust.rename(columns = {'Age' : 'age'}, inplace = True)

In [17]:
#check

df_cust.head()

,user_id,gender,state,age,date_joined,num_of_children,fam_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


In [18]:
#checking data types

df_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   user_id          206209 non-null  int64 
 1   gender           206209 non-null  object
 2   state            206209 non-null  object
 3   age              206209 non-null  int64 
 4   date_joined      206209 non-null  object
 5   num_of_children  206209 non-null  int64 
 6   fam_status       206209 non-null  object
 7   income           206209 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 12.6+ MB


No need to change any of the data types at this time.

In [25]:
#check make sure we still have same number of rows and columns after removing the columns and change the names

df_cust.shape

(206209, 8)

**2. Data Quality and Consistency Checks**

In [31]:
#check statistical data to see if anything is out of the ordinary

df_cust.describe()

,user_id,age,num_of_children,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


Nothing seems out of the ordinary here, user_id obviously does not need to be here, however, I believe this is the column which we will use to merge with the other data set.  Due to this reason, I do not want to change its data type just yet, I will change it if necessary for the merge, or perhaps after it.  

In [28]:
#check for mixed-type data in df_cust

for col in df_cust.columns.tolist():
  weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_cust[weird]) > 0:
    print (col)

There is no mix data in df_cust.

In [29]:
#check for missing values

df_cust.isnull().sum()

user_id            0
gender             0
state              0
age                0
date_joined        0
num_of_children    0
fam_status         0
income             0
dtype: int64

There are no missing values in df_cust.

In [33]:
#check for duplicates

df_cust[df_cust.duplicated()]

,user_id,gender,state,age,date_joined,num_of_children,fam_status,income


There are no duplicates in this df_cust.

**3. Merge consumer data with rest of Instacart data**

In [35]:
#check shape of ords_prods_merge

ords_prods_merge.shape

(32404859, 25)

It has the correct number of rows and columns.

In [37]:
#display all columns

pd.set_option('display.max_columns', None)

In [38]:
#check if any column needs to be removed

ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,customer_status,loyalty_flag,avg_user_price,spender_status,med_days_prior_order,frequency_status
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regular Days,Average orders,10,New customer,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [39]:
#remove _merge column to merge successfully with df_cust

ords_prods_merge = ords_prods_merge.drop(columns = ['_merge'])

In [40]:
#check _merge was removed

ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,customer_status,loyalty_flag,avg_user_price,spender_status,med_days_prior_order,frequency_status
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Regular Days,Average orders,10,New customer,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,New customer,6.367797,Low spender,20.5,Non-frequent customer


The _merge column from ords_prods_merge was succesfully removed.

In [41]:
#check data type for user_id in ords_prods_merge

ords_prods_merge['user_id'].info()

<class 'pandas.core.series.Series'>
Int64Index: 32404859 entries, 0 to 32404858
Series name: user_id
Non-Null Count     Dtype
--------------     -----
32404859 non-null  int64
dtypes: int64(1)
memory usage: 494.5 MB


On both data sets, the data type for user_id is int64, so we can go ahead with the merge now.

In [42]:
#merge the data sets as cust_ords_prods_merge, using outer to make sure it keeps all information from both data sets

cust_ords_prods_merge = pd.merge(ords_prods_merge, df_cust, on = 'user_id', how = 'outer', indicator = True)

In [43]:
#check if merge was successful

cust_ords_prods_merge['_merge'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

In [45]:
#check to make sure we have 24(from ords_prods_merge)  + 8 (df_cust) = 32 columns in our cust_ords_prods_merge

cust_ords_prods_merge.shape

(32404859, 32)

In [46]:
#check columns

cust_ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,customer_status,loyalty_flag,avg_user_price,spender_status,med_days_prior_order,frequency_status,gender,state,age,date_joined,num_of_children,fam_status,income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Regular Days,Average orders,10,New customer,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both


In [47]:
#drop _merge column from cust_ords_prods_merge

cust_ords_prods_merge = cust_ords_prods_merge.drop(columns = ['_merge'])

In [49]:
#check if removal was successful

cust_ords_prods_merge.shape

(32404859, 31)

We have one less column, so it was successful.

**4. Export new merged data frame as PKL**

In [50]:
#export cust_ords_prods_merge as PKL

cust_ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'cust_ords_prods_merge.pkl'))